In [1]:
%load_ext autoreload
%autoreload 2

import torch
from training.util import sgd

c:\Users\flose\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import experiments.base.cifar as cifar

batch_size = 4
trainloader = cifar.cifar10_trainloader("./data", batch_size=batch_size)
testloader = cifar.cifar10_testloader("./data", batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = torch.device("cpu")

In [10]:
from training.pp import MAP

layers = [
    ("drop-preresnet-20", (32, 3, 10, 0.1)),
    ("logsoftmax", ())
]

pp = MAP(layers)
pp.train_model(1, torch.nn.NLLLoss(), sgd(0.001), trainloader, batch_size, device)

KeyboardInterrupt: 